In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PowerTransformer

In [3]:
df_val = pd.read_csv(r'D:\ML\Finding_defect\Files\val.csv')
df_val.describe()

,Unnamed: 0,date,ns,nd,nf,entrophy,la,ld,lt,fix,ndev,age,nuc,exp,rexp,sexp,bug
count,7621.000000,7.621000e+03,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000,7621.000000
mean,14593.753313,1.309232e+09,1.093820,1.415431,2.297205,0.469900,55.855268,97.832568,3173.971264,0.275817,16.301666,41.839818,268.595066,1752.087653,25.456921,104.664349,0.463587
std,12938.301879,2.151168e+08,0.424313,2.834192,10.255450,3.379968,526.476021,4192.685432,10526.874257,0.446954,17.639223,132.280917,1097.734906,2254.963690,44.709164,147.104422,0.498705
min,1.000000,9.143996e+08,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,4099.000000,1.131658e+09,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,684.000000,0.000000,4.000000,0.547530,27.000000,160.000000,5.300796,15.000000,0.000000
50%,11216.000000,1.311849e+09,1.000000,1.000000,1.000000,0.000000,6.000000,3.000000,1580.000000,0.000000,10.000000,5.122199,94.000000,828.000000,13.331529,48.000000,0.000000
75%,21187.000000,1.466260e+09,1.000000,1.000000,2.000000,0.430827,26.000000,13.000000,3401.000000,1.000000,22.000000,30.271829,258.000000,2485.000000,28.512380,132.000000,1.000000
max,48726.000000,1.665283e+09,8.000000,108.000000,435.000000,142.740834,27241.000000,363587.000000,388666.000000,1.000000,209.000000,3741.089051,52411.000000,12172.000000,1088.711839,1240.000000,1.000000


In [4]:
df_val.drop(df_val.columns[0:3], axis =1, inplace= True)


In [5]:
for i in df_val.columns:
    percentile25 = df_val[i].quantile(0.25)
    percentile75 = df_val[i].quantile(0.75)
    iqr = percentile75 - percentile25
    upper_limit = percentile75 + 6 * iqr
    lower_limit = percentile25 - 6 * iqr
    df_val = df_val.drop(df_val.loc[(df_val[i] > upper_limit) | (df_val[i] < lower_limit)].index)

In [6]:
y_val = df_val['bug']
df_val.drop(['bug'], axis=1, inplace= True)
X_val = df_val
X_val.describe()

,ns,nd,nf,entrophy,la,ld,lt,fix,ndev,age,nuc,exp,rexp,sexp
count,4635.0,4635.0,4635.0,4635.0,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000,4635.000000
mean,1.0,1.0,1.0,0.0,7.713700,3.681769,1743.774973,0.302697,14.597411,19.128624,151.547357,1670.037109,18.700501,85.343905
std,0.0,0.0,0.0,0.0,11.211787,5.257085,1682.946826,0.459475,14.382070,34.247944,204.856578,2226.024574,22.165180,114.889936
min,1.0,1.0,1.0,0.0,0.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
25%,1.0,1.0,1.0,0.0,1.000000,1.000000,557.000000,0.000000,4.000000,0.185509,22.000000,144.000000,4.319703,12.000000
50%,1.0,1.0,1.0,0.0,3.000000,2.000000,1180.000000,0.000000,10.000000,3.314120,72.000000,768.000000,11.424821,41.000000
75%,1.0,1.0,1.0,0.0,9.000000,4.000000,2425.000000,1.000000,21.000000,20.454051,194.000000,2327.000000,22.616153,104.000000
max,1.0,1.0,1.0,0.0,78.000000,29.000000,13351.000000,1.000000,91.000000,185.389363,1253.000000,12172.000000,137.097634,674.000000


In [7]:
pt = PowerTransformer()
pX_val = pt.fit_transform(X_val)

In [8]:
sc = StandardScaler()
zX_val = sc.fit_transform(pX_val)

In [9]:
model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=10000, solver='saga')

In [10]:
param_grid = {
    'C' : [0.01, 0.1, 1],
    'penalty' : ['l1', 'l2','elasticnet'],
    'l1_ratio': [0.2, 0.5, 0.8]
}

In [11]:
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
}

In [12]:
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = 5, refit = 'roc_auc')
grid_search.fit(zX_val, y_val)

C:\Users\kinng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\kinng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\kinng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\kinng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packag

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=10000, random_state=42,
                                          solver='saga'),
             param_grid={'C': [0.01, 0.1, 1], 'l1_ratio': [0.2, 0.5, 0.8],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             refit='roc_auc',
             scoring={'accuracy': 'accuracy', 'f1': 'f1',
                      'precision': 'precision', 'recall': 'recall',
                      'roc_auc': 'roc_auc'})

In [13]:
best_params = grid_search.best_params_
best_params

{'C': 0.01, 'l1_ratio': 0.2, 'penalty': 'l1'}

In [14]:
best_score = grid_search.best_score_
best_score

np.float64(0.7030849520342803)